In [1]:
from pandas import datetime
from matplotlib import pyplot
 
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')
 
series = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
print(series.head())
series.plot()
pyplot.show()from pandas import datetime

from matplotlib import pyplot

 

def parser(x):

    return datetime.strptime('190'+x, '%Y-%m')

 

series = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

print(series.head())

series.plot()

pyplot.show()



SyntaxError: invalid syntax (<ipython-input-1-e0fd2af1c007>, line 10)